In [10]:
import sys
import numpy as np
import struct
from collections import namedtuple
from importAgilentBin import *
from matplotlib import pyplot as plt
import os
from scipy.integrate import trapz
import csv

In [11]:
def readfile(binFile, arg):
    """read binFile and return data as numpy array
    :binFile: path to bin file
    :arg: an int, indicate the nth waveform to read
    """
    selWav = 0
    headerFmt = "5if3d2i16s16s24s16sdI"
    headerSiz = struct.calcsize(headerFmt) 
    waveformHeader = namedtuple("waveformHeader",
        "headerSize waveformType nWaveformBuffers nPoints \
        count xDisplayRange xDisplayOrigin xIncrement \
        xOrigin xUnits yUnits dateString timeString \
        frameString waveformString timeTag segmentIndex")
    bufHeaderFmt = "ihhi"
    bufHeaderSiz = struct.calcsize(bufHeaderFmt)
    bufHeader = namedtuple("bufHeader",
        "headerSize bufferType bytesPerPoint bufferSize")

    data = None
    time = None

    try:
        fd = open(binFile, 'rb')
        magic, fileVer, fileSize, nWav = struct.unpack(
            '2s2sii', fd.read(12))

        if magic.decode('ascii') != "AG":
            sys.stderr.write("Unrecognized file format\n")
            return None, None

        if arg and arg <= nWav:
            selWav = arg

        for idx in range(nWav):
            # read waveform header
            header = waveformHeader._make(
                    struct.unpack(headerFmt, fd.read(headerSiz)))
            # skip remaining data in the header
            fd.seek(header.headerSize - headerSiz, 1)

            if idx == selWav:
                stop = header.xIncrement * header.nPoints + header.xOrigin
                time = np.linspace(header.xOrigin, stop, header.nPoints)

            for bufIdx in range(header.nWaveformBuffers):
                bufferHeader = bufHeader._make(
                        struct.unpack(bufHeaderFmt, fd.read(bufHeaderSiz)))
                if idx == selWav:
                    if bufferHeader.bufferType in [1, 2, 3]:
                        fmt = "%df" % (header.nPoints)
                    elif bufferHeader.bufferType == 4:
                        fmt = "%di" % (header.nPoints)
                    elif bufferHeader.bufferType == 5:
                        fmt = "%dB" % (header.nPoints)
                    else:
                        fmt = "%dB" % (header.bufferSize)
                    data = np.asarray(struct.unpack(
                        fmt, fd.read(struct.calcsize(fmt))))
                else:
                    # skip waveform
                    fd.seek(bufferHeader.bufferSize, 1)

        return time, data
    except IOError:
        sys.stderr.write("%s open error\n" & (binFile))
        return None, None
    finally:
        fd.close()


In [12]:
# data = np.memmap("lu17600000.bin", dtype='float32', mode='r')
# print(data.shape)
data = readfile("lu17600000.bin", 30)
np.savetxt('lu17600000.csv', data, delimiter=' ')

In [13]:
time, data = readfile("/home/nasajpl/Desktop/20230412jpl_capstone/tc99m/tc99m00000.bin", 5)
print(max(data))

0.02611425891518593


In [14]:
def count_files_in_folder(folder_path):
    file_count = 0

    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

In [15]:
x = count_files_in_folder("/home/nasajpl/Desktop/20230412jpl_capstone/tc99m")
print(x)

11000


In [16]:
def convert_to_csv(order_list, data_list, file_path):
    rows = zip(order_list, data_list)

    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Time', 'Data'])  # Write header row
        writer.writerows(rows)

In [36]:
# Ted's improved convert function
def readBinaryFile(root_path, src_name):
    folder_len = count_files_in_folder(root_path + "/" + src_name)
    print("Using data: "+ root_path + "/" + src_name)

    # our special case: na22
    src_file_name = src_name
    if src_name == "na22-wideEwin":
        src_file_name = "na22-wide"

    x_order = np.arange(0, folder_len)
    integrals = []
    
    for i in range(folder_len):
        suffix = str(i)
        while len(suffix) < 5:
            suffix = '0' + suffix
        path = root_path + "/" + src_name + "/" + src_file_name + suffix + ".bin"
        
        time, val = readfile(path,20)
        x = time
        y = val
        # print(x)

        # Visualize time-series data
        # plt.plot(x, y)
        # plt.xlabel('Second')
        # plt.ylabel('Volt')
        # plt.title('Volts vs. Seconds')
        # plt.show()

        # slice a time window
        x_window = []
        y_window = []
        for i in range(len(x)):
            if x[i] >= 4e-07: # unit in s
                x_window = x[:i + 1]
                y_window = y[:i + 1]
                break
        
        # calculate the integral
        interval_integral = trapz(y_window, x_window)
        integrals.append(interval_integral)
        # print(interval_integral)

    # write into csv files
    convert_to_csv(x_order, integrals, file_path=root_path + "/integrals/" + src_name + "_integrals.csv")
    return 

In [38]:
root_path = "/home/nasajpl/Desktop/20230412jpl_capstone"
readBinaryFile(root_path, src_name="tc99m")
readBinaryFile(root_path, src_name="lu176")
readBinaryFile(root_path, src_name="na22-wideEwin")

Using data: /home/nasajpl/Desktop/20230412jpl_capstone/tc99m
Using data: /home/nasajpl/Desktop/20230412jpl_capstone/lu176
Using data: /home/nasajpl/Desktop/20230412jpl_capstone/na22-wideEwin


In [6]:
# Kevin's method using confidence interval
import scipy.stats as st
print(maximum_values)
confMax, confMin = st.t.interval(.9, len(maximum_values)-1, np.mean(maximum_values), st.sem(maximum_values))
print(confMax, confMin)
valid = []
for i in range(10999):
    if (maximum_values[i]<confMax) and (maximum_values[i]>confMin):
        valid.append("valid")
    else: 
        valid.append("invalid")

print(valid)
        
arr = np.array([integrals, maximum_values])


[0.02611425891518593, 0.015199711546301842, 0.021931754425168037, 0.06278756260871887, 0.01647447980940342, 0.010364018380641937, 0.02155248448252678, 0.018212800845503807, 0.020498955622315407, 0.017506936565041542, 0.013967083767056465, 0.02011968567967415, 0.13902083039283752, 0.01652715541422367, 0.014241000637412071, 0.015189175494015217, 0.020857155323028564, 0.018160125240683556, 0.01511542871594429, 0.025071267038583755, 0.01508382335305214, 0.011712534353137016, 0.015747545287013054, 0.020309321582317352, 0.016737861558794975, 0.019845768809318542, 0.010069030337035656, 0.01069061178714037, 0.018181195482611656, 0.015684334561228752, 0.019466498866677284, 0.023996667936444283, 0.017549078911542892, 0.022395307198166847, 0.05932145565748215, 0.022363699972629547, 0.01645340956747532, 0.01462027058005333, 0.02163676545023918, 0.023195987567305565, 0.021215355023741722, 0.014356888830661774, 0.01596878655254841, 0.016737861558794975, 0.010648471303284168, 0.020404139533638954, 0.

In [7]:
import pandas as pd
data = {'integrals': integrals,
        'maximum_values': maximum_values, 'valid': valid}
df = pd.DataFrame(data)
df.to_csv('tcm_integrals.csv')

In [8]:
# import pandas as pd

# data = {}
# df = pd.DataFrame(data)

# for i in range(10999):
#     suffix = str(i)
#     while len(suffix) < 5:
#         suffix = '0' + suffix
    
#     df_row = pd.read_csv("/home/nasajpl/Desktop/20230412jpl_capstone/tc99m/tc99m" + suffix + ".csv", sep=',', header=None)
    
#     res = 1
#     for v in val:
#         if v > threshold:
#             res = 0
#             break
#     val.append(res)
#     print(res)
#     arr = np.array(val)
#     np.savetxt("/home/nasajpl/Desktop/20230412jpl_capstone/tc99m/tc99m" + suffix + ".csv", arr, delimiter=',')